<a href="https://colab.research.google.com/github/Gjeffroy/hyperparam_autotuning_keras/blob/main/MNIST_CNN_hyperparameter_tuning_witth_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install keras
! pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from keras_tuner.tuners import RandomSearch

# Define the model-building function
def build_model(hp):
    model = tf.keras.Sequential()

    # Tune the number of convolutional layers
    model.add(tf.keras.layers.Conv2D(filters=hp.Int('conv_1_filters', min_value=32, max_value=256, step=32),
                                     kernel_size=3,
                                     activation='relu',
                                     input_shape=(28, 28, 1)))

    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(tf.keras.layers.Conv2D(filters=hp.Int(f'conv_{i+2}_filters', min_value=32, max_value=256, step=32),
                                         kernel_size=3,
                                         activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

    model.add(tf.keras.layers.Flatten())

    # Tune the number of dense layers
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(tf.keras.layers.Dense(units=hp.Int(f'dense_{i}_units', min_value=32, max_value=512, step=32),
                                         activation='relu'))

    # Output layer
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Load and preprocess the dataset
def load_and_prepare_mnist():
    # Load the MNIST dataset
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

    # Normalize pixel values to between 0 and 1
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    # Reshape images to the format (batch_size, height, width, channels)
    train_images = train_images.reshape((-1, 28, 28, 1))
    test_images = test_images.reshape((-1, 28, 28, 1))

    return (train_images, train_labels), (test_images, test_labels)

def main():
    # Load and prepare the MNIST dataset
    (train_images, train_labels), (test_images, test_labels) = load_and_prepare_mnist()

    # Initialize tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=5,
        executions_per_trial=3,
        directory='my_dir',
        project_name='mnist_tuning'
    )

    # Perform the hyperparameter search
    tuner.search(train_images, train_labels, epochs=5, validation_split=0.1)

    # Get the best model
    best_model = tuner.get_best_models(num_models=1)[0]

    # Print the best hyperparameters
    print("\nBest Hyperparameters:")
    print(best_model.get_config())

    # Train the best model on the full training dataset
    print("\nTraining the best model...")
    best_model.fit(train_images, train_labels, epochs=10, validation_split=0.1)

    # Evaluate the best model on the test dataset
    print("\nEvaluating the best model on the test dataset...")
    loss, accuracy = best_model.evaluate(test_images, test_labels)
    print(f'Test accuracy: {accuracy}')

if __name__ == "__main__":
    main()


Trial 5 Complete [00h 03m 52s]
val_accuracy: 0.9907777905464172

Best val_accuracy So Far: 0.9907777905464172
Total elapsed time: 00h 18m 37s

Best Hyperparameters:
{'name': 'sequential', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 28, 28, 1), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'conv2d_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 28, 28, 1), 'filters': 128, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_nam

In [5]:
from kerastuner import HyperParameters
import os
import json

def summarize_tuner_attempts(directory):
    tuner_summaries = []

    # Iterate over each subdirectory in 'my_dir'
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)

        # Check if it's a directory
        if os.path.isdir(subdir_path):
            # Check if it contains a 'trial.json' file
            trial_file = os.path.join(subdir_path, 'trial.json')
            if os.path.exists(trial_file):
                # Load hyperparameters from 'trial.json'
                with open(trial_file, 'r') as f:
                    trial_data = json.load(f)
                hp = HyperParameters.from_config(trial_data['hyperparameters'])

                # Get the validation accuracy from 'trial.json'
                val_accuracy = trial_data.get('score')

                # Add hyperparameters and validation accuracy to the summaries list
                tuner_summaries.append((hp, val_accuracy))

    # Sort tuner summaries by the number of convolutional layers
    tuner_summaries.sort(key=lambda x: x[0].values['num_conv_layers'])

    return tuner_summaries

# Function to print CNN hyperparameters as a table
def print_hyperparameters_table(hp):
    print("Number of Convolutional Layers:", hp.values['num_conv_layers'])
    print("Number of Dense Layers:", hp.values['num_dense_layers'])
    print("Learning Rate:", hp.values['learning_rate'])
    print("\nCNN Hyperparameters:")
    sorted_conv_keys = sorted([key for key in hp.values.keys() if key.startswith('conv')])
    for key in sorted_conv_keys:
        print(f"| {key}: {hp.values[key]} |")
    print("\nDense Layer Hyperparameters:")
    sorted_dense_keys = sorted([key for key in hp.values.keys() if key.startswith('dense')])
    for key in sorted_dense_keys:
        print(f"| {key}: {hp.values[key]} |")
    print("\n")

# Function to print summary table
def print_summary_table(summary):
    print("Summary:")
    print("| Attempt | Accuracy | Num Conv Layers | Num Dense Layers |")
    print("|---------|----------|-----------------|-------------------|")
    for i, (hp, val_accuracy) in enumerate(summary, 1):
        num_conv_layers = hp.values['num_conv_layers']
        num_dense_layers = hp.values['num_dense_layers']
        print(f"| {i} | {val_accuracy} | {num_conv_layers} | {num_dense_layers} |")
    print("\n")

# Example usage:
summaries = summarize_tuner_attempts('my_dir/mnist_tuning')
print_summary_table(summaries)
for i, (hp, val_accuracy) in enumerate(summaries, 1):
    print(f"Attempt {i}:")
    print_hyperparameters_table(hp)
    print(f"Validation Accuracy: {val_accuracy}\n")


Summary:
| Attempt | Accuracy | Num Conv Layers | Num Dense Layers |
|---------|----------|-----------------|-------------------|
| 1 | 0.3964444448550542 | 2 | 2 |
| 2 | 0.9844444592793783 | 2 | 2 |
| 3 | 0.9907777905464172 | 3 | 2 |
| 4 | 0.3976111014684041 | 3 | 1 |
| 5 | 0.9891666571299235 | 3 | 3 |


Attempt 1:
Number of Convolutional Layers: 2
Number of Dense Layers: 2
Learning Rate: 0.01

CNN Hyperparameters:
| conv_1_filters: 64 |
| conv_2_filters: 32 |
| conv_3_filters: 256 |
| conv_4_filters: 160 |

Dense Layer Hyperparameters:
| dense_0_units: 96 |
| dense_1_units: 32 |


Validation Accuracy: 0.3964444448550542

Attempt 2:
Number of Convolutional Layers: 2
Number of Dense Layers: 2
Learning Rate: 0.01

CNN Hyperparameters:
| conv_1_filters: 160 |
| conv_2_filters: 32 |
| conv_3_filters: 32 |
| conv_4_filters: 224 |

Dense Layer Hyperparameters:
| dense_0_units: 32 |
| dense_1_units: 64 |


Validation Accuracy: 0.9844444592793783

Attempt 3:
Number of Convolutional Layers: 3
